# Un peu d'Exploratory Data Analysis avant toutes choses

Ce notebook vise à explorer rapidement la base de données pour avoir une idée de se composition, la répartition et éventuels déséquilibres entre les classes, les caractéristiques des textes à analyser. Cela est éventuellement l'occasion de détecter des erreurs de textes (encodage erroné au moment d'enregistrer/lire les demandes)

# 1. Packages usuels et importation de la base train

In [1]:
# Pour faciliter la mise à jour des fonctions écrites dans func_custom sans avoir à redémarrer le kernel
%load_ext autoreload
%autoreload 2

In [2]:
# Packages classiques
import pandas as pd

# Custom package
import func_custom as fc

# NLP
from wordcloud import WordCloud
from nltk.corpus import stopwords

# Dataviz
import seaborn as sns
import matplotlib.pyplot as plt

La variable `Unnamed: 0` semble juste être un indice de ligne, on peut pour le moment ne pas le garder pour l'analyse

In [3]:
# with open("data/data_train.xlsx", encoding = "utf-8") as f:
#     df_train = pd.read_excel(f, usecols = ["label", "message"])

In [4]:
df_train = pd.read_excel("data/data_train.xlsx",
                         usecols = ["label", "message"])

# 2. Preprocessing

In [5]:
# Générateur aléatoire de messages pour première inspection visuelle
message_test = df_train.sample(1)["message"].values[0]
message_test = df_train.iloc[7]["message"]
print(message_test)

bonjour j'ai changé d'employeur depuis le 0000er decembre0000. ma periode d'essai a été rompue par mon employeur, je finis donc le 0000/0000/0000 et serai au chomage à partir du 0000er mars 0000 comment ca va se passer pour les impots ,c'est moi qui payait pour notre couple ? reponse urgente merci XXXXX XXXXX mail XX@XX 0000


In [6]:
message_processed = fc.preprocess_text(message_test)

In [7]:
print(message_test)
print(message_processed)

bonjour j'ai changé d'employeur depuis le 0000er decembre0000. ma periode d'essai a été rompue par mon employeur, je finis donc le 0000/0000/0000 et serai au chomage à partir du 0000er mars 0000 comment ca va se passer pour les impots ,c'est moi qui payait pour notre couple ? reponse urgente merci XXXXX XXXXX mail XX@XX 0000
bonjour changé employeur depuis er decembr period essai été rompu employeur fin donc chomag à part er mar comment ca va pass impot pai coupl repons urgent merc mail


In [8]:
df_train["message_clean"] = df_train["message"].apply(lambda x : fc.preprocess_text(x))

In [9]:
df_train[df_train["label"] == "1"]["message_clean"]

5      bonjour suit dernier demand repons si percev p...
7      bonjour changé employeur depuis er decembr pe...
8      madam monsieur bonjour xxxxx́ san enfant à ch...
10     bonjour taux prélèv jour calculé revenus pr...
12     ten à signal depuis quelqu revenus encor bais...
                             ...                        
686    bonjour réuss à chang taux prélèv à sourc...
688    bonjour xxxxx́e dernièr travaill à temp part...
689    bonsoir actualis prélèv sourc car etais impo...
693    bonjour à jour déclaré comm divorcé suit a...
697    bonjour madam monsieur signal chang situat fam...
Name: message_clean, Length: 302, dtype: object

In [11]:
df_train.to_csv("data/data_clean.csv", 
                sep = ";",
                index = False)